# SPDE for Spiral Anisotropy

<!-- SUMMARY: Estimation and Simulations performed with a non-stationary anisotropy of the covariance (spiral form), treated in the SPDE formalism -->

<!-- CATEGORY: SPDE -->

In this tutorial, we show how the use of SPDE for Varying Anisotropy when this Anisotropy must follow a Spiral shape (defined as an external function)

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import random
import math

gdoc.setNoScroll()

Defining the Model as a single Matern structure. This function is defined as anisotropic: we clearly specify the extension of the ranges in the two main directions. The angle does not have to be defined here: it will be overwritten later as the non-stationary parameter.
Note that it is essential to define the short range of the anisotropy ellipsoid first (for the definition of angle as defined in the Spiral function used as a function)... otherwise future results will represent the shape otabined as the *orthogonal* of the spirale.

In [ ]:
model = gl.Model.createFromParam(gl.ECov.MATERN, 1., 1., 1., [4.,45.])

A Spiral function is defined and attached to the Model: this is a manner to update the Model by transforming the anisotropy angle as the *unique* non-stationary parameter.

In [ ]:
spirale = gl.FunctionalSpirale(0., -1.4, 1., 1., 50., 50.)
cova = model.getCovAniso(0)
cova.makeAngleNoStatFunctional(spirale)

A visualisation of the non-stationarity can be otanined in the following paragraph. The angle is represented at each node of a grid. For better legibility the grid is defined as a coarse grid.

In [ ]:
coarse = gl.DbGrid.create([26,26],[4.,4.])
res = gp.covaOnGrid(cova, coarse, scale=2000)

Creating a output grid

In [ ]:
grid = gl.DbGrid.create([101,101],[1.,1.]) 

Perform several non-conditional simulations on the grid, using the Model and the non-stationarity.

In [ ]:
nbsimu = 4
iuid = gl.simulateSPDE(None,grid,model,None,nbsimu)
grid

We represent the non-conditional simulations

In [ ]:
vmin = -4
vmax = +4
fig, ax = plt.subplots(2, 2, figsize=(16,12))
ax[0,0].raster(grid,name="SimuSPDE.1", useSel=False, vmin=vmin, vmax=vmax)
ax[0,1].raster(grid,name="SimuSPDE.2", useSel=False, vmin=vmin, vmax=vmax)
ax[1,0].raster(grid,name="SimuSPDE.3", useSel=False, vmin=vmin, vmax=vmax)
ax[1,1].raster(grid,name="SimuSPDE.4", useSel=False, vmin=vmin, vmax=vmax)
fig.subplots_adjust(right=0.7)

Extracting a set of nodes randomly located in order to create a data file which will serve as conditioning. The data is extracted from the first non-conditional simulation.

In [ ]:
data = gl.Db.createSamplingDb(grid, number=100, names=["x1", "x2", "SimuSPDE.1"])
data.setName("SimuSPDE.1", "data")
data

In [ ]:
res = gp.plot(data, nameColor="data")

Use the previous data set (and the non-stationary Model) in order to perform an estimation

In [ ]:
iuid = gl.krigingSPDE(data,grid,model)
grid

Representing the Estimation obtained on the Grid

In [ ]:
res = gp.plot(grid, "KrigingSPDE.data.estim")

Performing several conditional simulation

In [ ]:
nbsimu = 4
iuid = gl.simulateSPDE(data,grid,model,None,nbsimu, namconv=gl.NamingConvention("CondSimu"))
grid

Representing the conditional simulations

In [ ]:
vmin = -4
vmax = +4
fig, ax = plt.subplots(2,2,figsize=(16,12))
ax[0,0].raster(grid,name="CondSimu.*.1", useSel=False, vmin=vmin, vmax=vmax)
ax[0,1].raster(grid,name="CondSimu.*.2", useSel=False, vmin=vmin, vmax=vmax)
ax[1,0].raster(grid,name="CondSimu.*.3", useSel=False, vmin=vmin, vmax=vmax)
ax[1,1].raster(grid,name="CondSimu.*.4", useSel=False, vmin=vmin, vmax=vmax)

fig.subplots_adjust(right=0.7)

In [ ]:
model